In [1]:
!pip3 install pyspark

  Using cached https://files.pythonhosted.org/packages/86/ec/60880978512d5569ca4bf32b3b4d7776a528ecf4bca4523936c98c92a3c8/py4j-0.10.9.5-py2.py3-none-any.whl


In [1]:
import pyspark

In [2]:
import pandas as pd

In [5]:
!ls

derby.log  Pictures		  pig_1690366806721.log  pyspark.ipynb
Desktop    pig_1690307044527.log  pig_1690405518562.log  Templates
Documents  pig_1690307393867.log  pig_1690463400647.log  test.csv
Downloads  pig_1690363108423.log  pig_1690542314953.log  Videos
Music	   pig_1690366509798.log  Public


In [7]:
df=pd.read_csv('test.csv')
df.head()

,name,age,experience
0,aditya,34,10
1,meow,25,3
2,nigga,12,2


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName('practice').getOrCreate()

/home/printadi/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [5]:
spark

In [13]:
df_pyspark = spark.read.csv('test.csv',header=True)
df_pyspark

DataFrame[name: string, age: string, experience: string]

In [14]:
df_pyspark.show()

+------+---+----------+
|  name|age|experience|
+------+---+----------+
|aditya| 34|        10|
|  meow| 25|         3|
| nigga| 12|         2|
+------+---+----------+



In [15]:
df_pyspark.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- experience: string (nullable = true)



In [17]:
df_pyspark = spark.read.csv('test.csv',header=True, inferSchema=True) #using inferschema to get exact datatype of ach col in dataset
df_pyspark.show()
df_pyspark.printSchema()
#As you can see we got INTEGER in col AGE ; earlier we were getting String as AGE in dataset

+------+---+----------+
|  name|age|experience|
+------+---+----------+
|aditya| 34|        10|
|  meow| 25|         3|
| nigga| 12|         2|
+------+---+----------+

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)



In [18]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [23]:
print(df_pyspark.columns)

['name', 'age', 'experience']


In [25]:
#HOW TO SELECT A COLUMN
df_pyspark.select('name').show()

+------+
|  name|
+------+
|aditya|
|  meow|
| nigga|
+------+



In [26]:
#HOW TO SELECT multi COLUMN
df_pyspark.select(['name','age']).show()

+------+---+
|  name|age|
+------+---+
|aditya| 34|
|  meow| 25|
| nigga| 12|
+------+---+



In [27]:
#GET DATATYPES AND COLUMNS
df_pyspark.dtypes

[('name', 'string'), ('age', 'int'), ('experience', 'int')]

In [28]:
#GIVE SUMMARY ABOUT DATASET
df_pyspark.describe().show()

+-------+------+------------------+-----------------+
|summary|  name|               age|       experience|
+-------+------+------------------+-----------------+
|  count|     3|                 3|                3|
|   mean|  null|23.666666666666668|              5.0|
| stddev|  null| 11.06044001535804|4.358898943540674|
|    min|aditya|                12|                2|
|    max| nigga|                34|               10|
+-------+------+------------------+-----------------+



In [33]:
df_pyspark['age']

Column<'age'>

In [32]:
#Add columns in DATAFRAME
df_pyspark = df_pyspark.withColumn('Exp_after_2_yrs',df_pyspark['experience']+2)
df_pyspark.show()

+------+---+----------+---------------+
|  name|age|experience|Exp_after_2_yrs|
+------+---+----------+---------------+
|aditya| 34|        10|             12|
|  meow| 25|         3|              5|
| nigga| 12|         2|              4|
+------+---+----------+---------------+



In [35]:
#Drop the columns
df_pyspark = df_pyspark.drop('Exp_after_2_yrs')
df_pyspark.show()

+------+---+----------+
|  name|age|experience|
+------+---+----------+
|aditya| 34|        10|
|  meow| 25|         3|
| nigga| 12|         2|
+------+---+----------+



In [38]:
#Rename columns
df_pyspark.withColumnRenamed('name','NewName').show()

#OR USE
#df_pyspark = df_pyspark.withColumnRenamed('name','NewName')
#df_pyspark.show()

+-------+---+----------+
|NewName|age|experience|
+-------+---+----------+
| aditya| 34|        10|
|   meow| 25|         3|
|  nigga| 12|         2|
+-------+---+----------+



In [41]:
df = spark.read.csv('miss.csv',header=True,inferSchema=True)
df.show()

+------+----+----+
|  name| age| exp|
+------+----+----+
|aditya|  24|   2|
|  meow|null|   3|
|  null|  14|   3|
| nigga|null|null|
|  null|null|null|
+------+----+----+



In [ ]:
#HANDLING MISSING VALUES


In [42]:
#DROP THOSE ROWS WHICH ARE HAVING NULL VALUES
df.na.drop().show()

#OR YOU CAN USE
#df = df.na.drop()
#df.show()

+------+---+---+
|  name|age|exp|
+------+---+---+
|aditya| 24|  2|
+------+---+---+



In [43]:
#how parameter in drop
#drop(how = "", thres="", subset="") ---> these are the parameters we can give inside drop function while dealing NULL(na) values

#how parameter is going to tell python that , how we are going to delete null values, should we delete any row with 
#having even single NULL values or somthing else
df.na.drop(how="any").show() #--> "any" defines that if we have even 1 null values , then we should delete entire row

+------+---+---+
|  name|age|exp|
+------+---+---+
|aditya| 24|  2|
+------+---+---+



In [44]:
df.na.drop(how="all").show() # --> It is going to delete only those values which are having ALL null values in ROWS

+------+----+----+
|  name| age| exp|
+------+----+----+
|aditya|  24|   2|
|  meow|null|   3|
|  null|  14|   3|
| nigga|null|null|
+------+----+----+



In [46]:
#drop(thresh = "") --> It stands for threshold --> It says that MINIMUM NUMBER OF Non Null VALUES should be present 
#in row , if the some rows has less Non NULL values than threshold then we will drop it.

df.na.drop(thresh=1).show()

+------+----+----+
|  name| age| exp|
+------+----+----+
|aditya|  24|   2|
|  meow|null|   3|
|  null|  14|   3|
| nigga|null|null|
+------+----+----+



In [47]:
#Subset --> drop(subset=['colname']) --> THis basically says that on which colummn(s) we want to apply drop function,
#instead of applying drop() function on entire dataframe

df.na.drop(subset=['age']).show()

+------+---+---+
|  name|age|exp|
+------+---+---+
|aditya| 24|  2|
|  null| 14|  3|
+------+---+---+



In [53]:
#Fill up the Missing values --> This will replace all empty fields(null/na values) in dataframe with a given value
#fill('missing value', ['col1', 'col2'....])


df.na.fill('missingvalss',['name','age','exp']).show()

+------------+----+----+
|        name| age| exp|
+------------+----+----+
|      aditya|  24|   2|
|        meow|null|   3|
|missingvalss|  14|   3|
|       nigga|null|null|
|missingvalss|null|null|
+------------+----+----+



In [55]:
#Fill null values by the help of imputer

from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['age'],
    outputCols=['imputed_age']
).setStrategy("mean")

#Add imputation cols to df
imputer.fit(df).transform(df).show()

#If you dont want to make extra imputed col , rather than you want to show imputed values in the same cols then:-
#Y0U can keep inputCols and outputCols same
'''
imputer = Imputer(
    inputCols=['age'],
    outputCols=['age']
).setStrategy("mean")
'''

+------+---+----+
|  name|age| exp|
+------+---+----+
|aditya| 24|   2|
|  meow| 19|   3|
|  null| 14|   3|
| nigga| 19|null|
|  null| 19|null|
+------+---+----+



In [56]:
imputer = Imputer(
    inputCols=['exp'],
    outputCols=['imputed_exp']
).setStrategy("median")

imputer.fit(df).transform(df).show()

+------+----+----+-----------+
|  name| age| exp|imputed_exp|
+------+----+----+-----------+
|aditya|  24|   2|          2|
|  meow|null|   3|          3|
|  null|  14|   3|          3|
| nigga|null|null|          3|
|  null|null|null|          3|
+------+----+----+-----------+



In [ ]:
#FILTER OPERATIONS
# &,|,==
# ~

In [58]:
df1 = spark.read.csv('salary.csv', header=True, inferSchema=True)
df1.show()

+------+---+---+------+
|  name|age|exp|salary|
+------+---+---+------+
|aditya| 24|  4|400000|
|  meow| 13|  3|   400|
| nigga| 45|  3| 50000|
|   dom| 67| 39| 20000|
|chmaar| 20|  2| 12000|
+------+---+---+------+



In [59]:
#showing the records where salary<=20000

df1.filter("salary<=20000").show()

+------+---+---+------+
|  name|age|exp|salary|
+------+---+---+------+
|  meow| 13|  3|   400|
|   dom| 67| 39| 20000|
|chmaar| 20|  2| 12000|
+------+---+---+------+



In [60]:
df1.filter("salary<=20000").select(['name','exp']).show()

+------+---+
|  name|exp|
+------+---+
|  meow|  3|
|   dom| 39|
|chmaar|  2|
+------+---+



In [62]:
df1.filter( (df1['salary']>=5000) & (df1['salary']<=20000) ).show()

+------+---+---+------+
|  name|age|exp|salary|
+------+---+---+------+
|   dom| 67| 39| 20000|
|chmaar| 20|  2| 12000|
+------+---+---+------+



In [65]:
df1.filter(df1['name']=="aditya").show()

+------+---+---+------+
|  name|age|exp|salary|
+------+---+---+------+
|aditya| 24|  4|400000|
+------+---+---+------+



In [64]:
df1.filter(df1['name']=="aditya").select('salary').show()

+------+
|salary|
+------+
|400000|
+------+



In [66]:
df1.filter( (df1['name'] == "meow") | (df1['name'] == "aditya") ).show()

+------+---+---+------+
|  name|age|exp|salary|
+------+---+---+------+
|aditya| 24|  4|400000|
|  meow| 13|  3|   400|
+------+---+---+------+



In [68]:
df1.filter( ~(df1['name'] == "meow") ).show()

+------+---+---+------+
|  name|age|exp|salary|
+------+---+---+------+
|aditya| 24|  4|400000|
| nigga| 45|  3| 50000|
|   dom| 67| 39| 20000|
|chmaar| 20|  2| 12000|
+------+---+---+------+



In [7]:
df2 = spark.read.csv('test3.csv',header=True,inferSchema=True)
df2.show()

+----------+-----------+------+
|      name|departments|salary|
+----------+-----------+------+
|     krish|datascience| 10000|
|     krish|        iot|  5000|
|    mahesh|    bigdata|  4000|
|     krish|    bigdata|  4000|
|    mahesh|datascience|  3000|
|shudhanshu|datascience| 20000|
|shudhanshu|        iot| 10000|
|shudhanshu|    bigdata|  5000|
|     sunny|datascience| 10000|
|     sunny|    bigdata|  2000|
+----------+-----------+------+



In [8]:
df2.printSchema()

root
 |-- name: string (nullable = true)
 |-- departments: string (nullable = true)
 |-- salary: integer (nullable = true)



In [11]:
#GROUP BY OPERATIONS
#NOTE -> ALWAYS AGGREGATE FUNCTIONS WORKS WITH GROUPBY , AGGREGATED FUNCTION CAN'T BE CALLED ALONE (WITHOUT GROUPBY)
#So 1st we need to apply groupBy function then we need to apply aggregate function


df2.groupBy('name').sum().show()

+----------+-----------+
|      name|sum(salary)|
+----------+-----------+
|shudhanshu|      35000|
|     sunny|      12000|
|     krish|      19000|
|    mahesh|       7000|
+----------+-----------+



In [18]:
df2.groupBy('name').max().show()

+----------+-----------+
|      name|max(salary)|
+----------+-----------+
|shudhanshu|      20000|
|     sunny|      10000|
|     krish|      10000|
|    mahesh|       4000|
+----------+-----------+



In [20]:
df2.groupBy('departments').min().show()

+-----------+-----------+
|departments|min(salary)|
+-----------+-----------+
|        iot|       5000|
|    bigdata|       2000|
|datascience|       3000|
+-----------+-----------+



In [13]:
df2.groupBy('departments').sum().show()

+-----------+-----------+
|departments|sum(salary)|
+-----------+-----------+
|        iot|      15000|
|    bigdata|      15000|
|datascience|      43000|
+-----------+-----------+



In [19]:
df2.groupBy('departments').mean().show()

+-----------+-----------+
|departments|avg(salary)|
+-----------+-----------+
|        iot|     7500.0|
|    bigdata|     3750.0|
|datascience|    10750.0|
+-----------+-----------+



In [15]:
df2.groupBy('departments').count().show()

+-----------+-----+
|departments|count|
+-----------+-----+
|        iot|    2|
|    bigdata|    4|
|datascience|    4|
+-----------+-----+



In [ ]:
#PySparkML

In [22]:
df3 = spark.read.csv('test4.csv', header=True, inferSchema=True)
df3.show()

+----------+---+----------+------+
|      name|age|experience|salary|
+----------+---+----------+------+
|     krish| 31|        10| 30000|
|shudhanshu| 30|         8| 25000|
|     sunny| 29|         4| 20000|
|      paul| 24|         3| 20000|
|    harsha| 21|         1| 15000|
|   shubham| 23|         2| 18000|
+----------+---+----------+------+



In [23]:
df3.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [24]:
df3.columns

['name', 'age', 'experience', 'salary']

In [27]:
df3.dtypes

[('name', 'string'), ('age', 'int'), ('experience', 'int'), ('salary', 'int')]

In [35]:
#Now we are trying to create a ML model where we take age & experience as independent varibale and salary as
#dependent variable

from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=["age","experience"], outputCol="IndependentFeature")

In [36]:
output=featureassembler.transform(df3)

In [37]:
output.show()

+----------+---+----------+------+------------------+
|      name|age|experience|salary|IndependentFeature|
+----------+---+----------+------+------------------+
|     krish| 31|        10| 30000|       [31.0,10.0]|
|shudhanshu| 30|         8| 25000|        [30.0,8.0]|
|     sunny| 29|         4| 20000|        [29.0,4.0]|
|      paul| 24|         3| 20000|        [24.0,3.0]|
|    harsha| 21|         1| 15000|        [21.0,1.0]|
|   shubham| 23|         2| 18000|        [23.0,2.0]|
+----------+---+----------+------+------------------+



In [38]:
output.columns

['name', 'age', 'experience', 'salary', 'IndependentFeature']

In [39]:
finalizeddata = output.select(["IndependentFeature","salary"])
finalizeddata.show()

+------------------+------+
|IndependentFeature|salary|
+------------------+------+
|       [31.0,10.0]| 30000|
|        [30.0,8.0]| 25000|
|        [29.0,4.0]| 20000|
|        [24.0,3.0]| 20000|
|        [21.0,1.0]| 15000|
|        [23.0,2.0]| 18000|
+------------------+------+



In [41]:
from pyspark.ml.regression import LinearRegression
#test train split
traindata,testdata=finalizeddata.randomSplit([0.75, 0.25])
regressor=LinearRegression(featuresCol='IndependentFeature', labelCol='salary')
regressor=regressor.fit(traindata)

In [42]:
#coefficients
regressor.coefficients

DenseVector([47.619, 1285.7143])

In [43]:
#intercept
regressor.intercept

13619.047619047662

In [47]:
#predictions
pred_results = regressor.evaluate(testdata)
pred_results.predictions.show()

+------------------+------+-----------------+
|IndependentFeature|salary|       prediction|
+------------------+------+-----------------+
|        [23.0,2.0]| 18000|17285.71428571428|
|       [31.0,10.0]| 30000|27952.38095238097|
+------------------+------+-----------------+



/home/printadi/.local/lib/python3.6/site-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [49]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(1380.9523809523762, 2351473.922902466)

In [ ]:
#NEW DATASET

In [51]:
df4 = spark.read.csv('tips.csv', header=True, inferSchema=True)
df4.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [52]:
df4.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [53]:
df4.columns

['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [54]:
#handling categorical features --> sex, smoker, day, time --> we are doing this to convert string values to 
#integer for ML

#In ML we use oneHotEncoding but here in pySpark we will use StringIndexer


from pyspark.ml.feature import StringIndexer

In [56]:
indexer = StringIndexer(
    inputCols=["sex","smoker","day","time"],
    outputCols=["sex_indexed","smoker_indexed","day_indexed","tme_indexed"]
)
df_r = indexer.fit(df4).transform(df4)


In [57]:
df_r.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|tme_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|        0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|        0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|        0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|        0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|        0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|        0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|        0.0|
|     26.8

In [58]:
df_r.columns

['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_indexed',
 'day_indexed',
 'tme_indexed']

In [62]:
#NOW WE WILL USE VectorAssembler -> beacause we use Vector Assembler to group INDEPENDENT FEATURES together, 
#and we want dependent features to be separate

from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(
    inputCols=["tip","sex_indexed","smoker_indexed","day_indexed","tme_indexed","size"],
    outputCol="IndependentFeatures"
)
#Incase of SINGLE INPUT/INDEPENDENT feature use--> inputCol="column name"
#Incase of Multiple INPUT/INDEPENDENT feature use--> inputCols=["column name 1","column name 2","column name 3".....]
#Incase of SINGLE OUTPUT/DEPENDENT feature use--> outputCol="column name"
#Incase of Multiple OUTPUT/DEPENDENT feature use--> outputCols=["column name 1","column name 2","column name 3".....]

output = featureassembler.transform(df_r)

In [63]:
output.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+-----------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|tme_indexed| IndependentFeatures|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+-----------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|        0.0|[1.01,1.0,0.0,1.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|        0.0|[1.66,0.0,0.0,1.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|        0.0|[3.5,0.0,0.0,1.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|        0.0|[3.31,0.0,0.0,1.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|        0.0|[3.61,1.0,0.0,1.0...|
|     25.29|4.71|  Male|    No|Sun|Dinne

In [66]:
#NOW WE WILL ONLY SELECT IndependentFeatures as X and total_bill as Dependent feature(Y)

finalized_data = output.select(["IndependentFeatures", "total_bill"])

In [67]:
finalized_data.show()

+--------------------+----------+
| IndependentFeatures|total_bill|
+--------------------+----------+
|[1.01,1.0,0.0,1.0...|     16.99|
|[1.66,0.0,0.0,1.0...|     10.34|
|[3.5,0.0,0.0,1.0,...|     21.01|
|[3.31,0.0,0.0,1.0...|     23.68|
|[3.61,1.0,0.0,1.0...|     24.59|
|[4.71,0.0,0.0,1.0...|     25.29|
|[2.0,0.0,0.0,1.0,...|      8.77|
|[3.12,0.0,0.0,1.0...|     26.88|
|[1.96,0.0,0.0,1.0...|     15.04|
|[3.23,0.0,0.0,1.0...|     14.78|
|[1.71,0.0,0.0,1.0...|     10.27|
|[5.0,1.0,0.0,1.0,...|     35.26|
|[1.57,0.0,0.0,1.0...|     15.42|
|[3.0,0.0,0.0,1.0,...|     18.43|
|[3.02,1.0,0.0,1.0...|     14.83|
|[3.92,0.0,0.0,1.0...|     21.58|
|[1.67,1.0,0.0,1.0...|     10.33|
|[3.71,0.0,0.0,1.0...|     16.29|
|[3.5,1.0,0.0,1.0,...|     16.97|
|(6,[0,5],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [71]:
#Now Creating ML model---> LINEAR REGRESSION

from pyspark.ml.regression import LinearRegression
#train test split
traindata,testdata = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='IndependentFeatures', labelCol='total_bill')
regressor = regressor.fit(traindata)

In [72]:
pred_result = regressor.evaluate(testdata)

In [74]:
pred_result.predictions.show()

+--------------------+----------+------------------+
| IndependentFeatures|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,5],[1.47,2.0])|     10.77|13.003528728897463|
|(6,[0,5],[2.72,2.0])|     13.28| 16.85676115986729|
|(6,[0,5],[3.18,2.0])|     19.82| 18.27475069446419|
|(6,[0,5],[3.27,2.0])|     17.78|18.552183429494015|
| (6,[0,5],[3.6,3.0])|     24.06|23.059536143667486|
|(6,[0,5],[3.76,2.0])|     18.24|20.062650542434188|
|(6,[0,5],[4.08,2.0])|     17.92|21.049078044762464|
|(6,[0,5],[5.92,3.0])|     29.03|30.211135535547484|
|(6,[0,5],[6.73,4.0])|     48.27|36.198129503213366|
|[1.1,1.0,1.0,0.0,...|      12.9| 12.73304063049877|
|[1.25,1.0,0.0,2.0...|      8.51| 9.573035965406785|
|[1.5,1.0,0.0,0.0,...|     26.41|11.619524928276986|
|[1.5,1.0,0.0,2.0,...|     10.65|10.343682451600749|
|[1.58,0.0,1.0,3.0...|     13.42|14.635264877754622|
|[1.73,0.0,0.0,2.0...|      9.78|12.529158597862951|
|[1.92,0.0,1.0,3.0...|      8.58| 12.193244746

/home/printadi/.local/lib/python3.6/site-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [85]:
#OTHER VALUES MSE, MAE, COEFFICENTS, INTERCEPT
print(pred_result.meanSquaredError)
print(pred_result.meanAbsoluteError)
print(regressor.coefficients)
print(regressor.intercept)
print(pred_result.r2)

29.986981178559905
3.8569851906934702
[3.0825859447758623,-1.4764813789637525,2.3465500801321277,0.2219440914759227,-1.7197306596280817,3.4900993523974346]
1.4919286852820755
0.6474116498615599
